<a href="https://colab.research.google.com/github/MH-1294/ML_DL/blob/main/RRS_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()


Saving Experiment_project.xlsx to Experiment_project.xlsx


In [3]:
import pandas as pd

# Replace with your file name (after upload)
file_name = list(uploaded.keys())[0]

# Read all sheets first
xls = pd.ExcelFile(file_name)
print("Available sheets:", xls.sheet_names)

# Load the first sheet (or whichever you need)
df = pd.read_excel(file_name, sheet_name=xls.sheet_names[0])

# Show first few rows and columns
print("Columns:", df.columns.tolist())
print(df.head())


Available sheets: ['Sheet1']
Columns: ['Eps', 'Method', 'Region', 'Clean mIoU', 'Adv mIoU']
   Eps           Method  Region  Clean mIoU  Adv mIoU
0  0.0          Regular    Left      0.8318    0.8318
1  NaN          Regular   Right      0.8318    0.8318
2  NaN          Regular  Center      0.8318    0.8318
3  NaN          Regular    Full      0.8318    0.8318
4  NaN  Label_Smoothing    Left      0.8605    0.8605


In [4]:
# Group by Eps and Method, then compute RRS across regions
rrs = (
    df.groupby(["Eps", "Method"])
      .apply(lambda g: (g["Adv mIoU"] / g["Clean mIoU"]).mean())
      .reset_index(name="RRS")
)

print(rrs)


    Eps   Method       RRS
0  0.00  Regular  1.000000
1  0.03  Regular  0.751367
2  0.06  Regular  0.685524
3  0.10  Regular  0.662539


/tmp/ipython-input-917310477.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g["Adv mIoU"] / g["Clean mIoU"]).mean())


In [5]:
rrs_table = rrs.pivot(index="Eps", columns="Method", values="RRS")
print(rrs_table.round(3))  # round for readability


Method  Regular
Eps            
0.00      1.000
0.03      0.751
0.06      0.686
0.10      0.663


In [6]:
print(df.columns.tolist())


['Eps', 'Method', 'Region', 'Clean mIoU', 'Adv mIoU']


In [7]:
print(df.groupby(["Eps", "Method"]).size())


Eps   Method 
0.00  Regular    1
0.03  Regular    1
0.06  Regular    1
0.10  Regular    1
dtype: int64


In [8]:
df["Method"] = df["Method"].astype(str).str.strip()
print(df["Method"].unique())


['Regular' 'Label_Smoothing' 'CutMix' 'CRAT' 'BlurPool']


In [11]:
print(df[df["Eps"].isna()])


    Eps           Method  Region  Clean mIoU  Adv mIoU
1   NaN          Regular   Right      0.8318    0.8318
2   NaN          Regular  Center      0.8318    0.8318
3   NaN          Regular    Full      0.8318    0.8318
4   NaN  Label_Smoothing    Left      0.8605    0.8605
5   NaN  Label_Smoothing   Right      0.8605    0.8605
..  ...              ...     ...         ...       ...
75  NaN             CRAT    Full      0.7506    0.7082
76  NaN         BlurPool    Left      0.8318    0.5511
77  NaN         BlurPool   Right      0.8318    0.5643
78  NaN         BlurPool  Center      0.8318    0.5278
79  NaN         BlurPool    Full      0.8318    0.2769

[76 rows x 5 columns]


In [12]:
df["Eps"] = df["Eps"].fillna(method="ffill")


/tmp/ipython-input-3616417508.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["Eps"] = df["Eps"].fillna(method="ffill")


In [13]:
print(df.groupby(["Eps","Method"]).size())


Eps   Method         
0.00  BlurPool           4
      CRAT               4
      CutMix             4
      Label_Smoothing    4
      Regular            4
0.03  BlurPool           4
      CRAT               4
      CutMix             4
      Label_Smoothing    4
      Regular            4
0.06  BlurPool           4
      CRAT               4
      CutMix             4
      Label_Smoothing    4
      Regular            4
0.10  BlurPool           4
      CRAT               4
      CutMix             4
      Label_Smoothing    4
      Regular            4
dtype: int64


In [14]:
import pandas as pd

# -----------------------------
# 1. Load Excel file
# -----------------------------
file_name = "Experiment_project.xlsx"   # replace with your actual uploaded file
df = pd.read_excel(file_name)

# -----------------------------
# 2. Clean column names + strip spaces
# -----------------------------
df = df.rename(columns=lambda x: x.strip())

# Ensure Eps and Method are clean
df["Method"] = df["Method"].astype(str).str.strip()
df["Eps"] = pd.to_numeric(df["Eps"], errors="coerce")

# -----------------------------
# 3. Fix NaN in Eps by forward-filling
# -----------------------------
df["Eps"] = df["Eps"].fillna(method="ffill")

# -----------------------------
# 4. Ensure numeric columns
# -----------------------------
df["Clean mIoU"] = pd.to_numeric(df["Clean mIoU"], errors="coerce")
df["Adv mIoU"]   = pd.to_numeric(df["Adv mIoU"], errors="coerce")

# -----------------------------
# 5. Compute RRS
# -----------------------------
rrs = (
    df.groupby(["Eps", "Method"])
      .apply(lambda g: (g["Adv mIoU"] / g["Clean mIoU"]).mean())
      .reset_index(name="RRS")
)

# Pivot into table format
rrs_table = rrs.pivot(index="Eps", columns="Method", values="RRS")
rrs_table = rrs_table.round(3)

print("Final Relative Robustness Score (RRS) Table:")
print(rrs_table)

# -----------------------------
# 6. Export to LaTeX (optional)
# -----------------------------
latex_table = rrs_table.to_latex(
    index=True,
    caption="Relative Robustness Score (RRS) across perturbation strengths.",
    label="tab:rrs",
    bold_rows=True
)
print(latex_table)

# Save if needed
with open("rrs_table.tex", "w") as f:
    f.write(latex_table)


✅ Final Relative Robustness Score (RRS) Table:
Method  BlurPool   CRAT  CutMix  Label_Smoothing  Regular
Eps                                                      
0.00       1.000  1.000   1.000            1.000    1.000
0.03       0.813  0.960   0.988            0.847    0.686
0.06       0.706  1.018   0.991            0.759    0.610
0.10       0.577  0.967   0.606            0.743    0.577
\begin{table}
\caption{Relative Robustness Score (RRS) across perturbation strengths.}
\label{tab:rrs}
\begin{tabular}{lrrrrr}
\toprule
Method & BlurPool & CRAT & CutMix & Label_Smoothing & Regular \\
Eps &  &  &  &  &  \\
\midrule
\textbf{0.000000} & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 \\
\textbf{0.030000} & 0.813000 & 0.960000 & 0.988000 & 0.847000 & 0.686000 \\
\textbf{0.060000} & 0.706000 & 1.018000 & 0.991000 & 0.759000 & 0.610000 \\
\textbf{0.100000} & 0.577000 & 0.967000 & 0.606000 & 0.743000 & 0.577000 \\
\bottomrule
\end{tabular}
\end{table}



/tmp/ipython-input-2925950058.py:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["Eps"] = df["Eps"].fillna(method="ffill")
/tmp/ipython-input-2925950058.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g["Adv mIoU"] / g["Clean mIoU"]).mean())
